In [ ]:
import pandas as pd
import os
import numpy as np
import random
import sys
sys.path.append('/home/graduates/cyn/Cell2Map')
from Cell2Map import common
import matplotlib.pyplot as plt
import scanpy as sc
import sys

from Cell2Map import map_utils
from Cell2Map import autoencoder
from Cell2Map import utils
import anndata as ad

# Load data

Read dataset

In [354]:
scRNA_path="../data/DCIS2/sc_RNA_DCIS2.h5ad"
cell_type_path=None
st_path="../data/DCIS2/spatial_DCIS2.h5ad"
coordinates_path=None
output_folder="../result/DCIS2/"
mean_cell_numbers=5

In [355]:
celltype_col='cell_type'

# Read the data from the specified paths
# sc_adata: single-cell RNA-seq data
# spatial_adata: spatial transcriptomics data
# cell_type_number_eachspot_data: estimated number of each cell type in each spot
# coordinates_data: coordinates of each spot
sc_adata, spatial_adata, coordinates_data =\
        common.read_data(scRNA_path, cell_type_path, st_path, coordinates_path,celltype_col)

# Extract the cell type information from the single-cell data
# Rename the index and the 'CellType' column for clarity
cell_type=pd.DataFrame(sc_adata.obs[celltype_col])
cell_type.index = [str(idx)[5:] for idx in cell_type.index]
cell_type['CellType'] = [str(idx)[5:] for idx in cell_type[celltype_col]]

# estimate the number of cells in each spot
cell_number_to_node_assignment=common.estimate_cell_number_RNA_reads(spatial_adata.to_df().T, int(mean_cell_numbers))

In [356]:
sc_adata,spatial_adata=map_utils.process_adatas(sc_adata,spatial_adata)

/home/graduates/anaconda3/envs/celloc/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:207: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [357]:
print(sc_adata)
print(spatial_adata)

AnnData object with n_obs × n_vars = 3587 × 19840
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'RNA_snn_res.1', 'seurat_clusters', 'cell_type', 'CellType'
    var: 'n_cells'
    uns: 'log1p', 'pca'
    obsm: 'X_pca'
    varm: 'PCs'
AnnData object with n_obs × n_vars = 1567 × 19840
    obs: 'in_tissue', 'array_row', 'array_col', 'rna_count_based_density'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'
    uns: 'spatial', 'log1p'
    obsm: 'spatial'


In [358]:
sc_adata,spatial_adata=autoencoder.embedding_feature(sc_adata,spatial_adata,k_cutoff=5)

The graph contains 26051 edges, 3587 cells.
7.262614998606077 neighbors per cell on average.
The graph contains 0 edges, 1567 cells.
0.0 neighbors per cell on average.


100%|██████████| 500/500 [00:10<00:00, 48.77it/s]


In [359]:
Deep_map=map_utils.map_cell_to_space(sc_adata,spatial_adata,
learning_rate=0.005,
    num_epochs=1800,
    lambda_d=1.22,
    lambda_g1=1.75,
    lambda_g2=0.78,
    b_init=cell_number_to_node_assignment,
    alpha=2.05,
    lambda_mahalanobis=0.65,
    lambda_distance=0.012,
    device='cuda:0')

/home/graduates/cyn/Cell2spotGATE/Cell2spotGATE/map_optimizer.py:50: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.d=torch.tensor(d,device=device, dtype=torch.float32)


seed:1000
Total_loss: 12.339, gv_term: 1.373, vg_term: 0.729, density_term: 8.976, expression_term: 6.886, distance_term:1.421,
Total_loss: 11.947, gv_term: 1.437, vg_term: 0.729, density_term: 9.003, expression_term: 6.520, distance_term:1.411,
Total_loss: 11.648, gv_term: 1.437, vg_term: 0.729, density_term: 9.068, expression_term: 6.155, distance_term:1.410,
Total_loss: 11.435, gv_term: 1.394, vg_term: 0.729, density_term: 9.156, expression_term: 5.814, distance_term:1.412,
Total_loss: 11.288, gv_term: 1.332, vg_term: 0.729, density_term: 9.257, expression_term: 5.510, distance_term:1.418,
Total_loss: 11.183, gv_term: 1.270, vg_term: 0.729, density_term: 9.364, expression_term: 5.246, distance_term:1.428,
Total_loss: 11.101, gv_term: 1.212, vg_term: 0.730, density_term: 9.479, expression_term: 5.010, distance_term:1.448,
Total_loss: 11.022, gv_term: 1.157, vg_term: 0.730, density_term: 9.614, expression_term: 4.785, distance_term:1.490,
Total_loss: 10.936, gv_term: 1.105, vg_term: 0

In [360]:
sc_sp_map_df = pd.DataFrame(Deep_map.X, index=sc_adata.obs_names, columns=spatial_adata.obs_names)


In [361]:
# Remove the first 5 characters from the column and index names of the DataFrame
sc_sp_map_df.columns = [str(col)[5:] for col in sc_sp_map_df.columns]
sc_sp_map_df.index = [str(idx)[5:] for idx in sc_sp_map_df.index]

# Initialize an empty dictionary to store the results
result_dict = {}

# For each column in the DataFrame, find the cell names with the highest values
# The number of cell names to find is determined by cell_number_to_node_assignment
# Store the results in the result_dict dictionary
for i, column in enumerate(sc_sp_map_df.columns.values):
    result_dict[column] = sc_sp_map_df[column].nlargest(cell_number_to_node_assignment[i]).index

# Convert the result_dict dictionary to a DataFrame
# The 'Predict' column contains the spot names, and the 'Values' column contains the cell names
# The 'Values' column is exploded so that each cell name has its own row
max_column_names = pd.DataFrame({'Predict': list(result_dict.keys()), 'Values': [list(index_obj) for index_obj in result_dict.values()]}).explode('Values')

# Convert the cell names in the 'Values' column to strings
max_column_names['Values'] = max_column_names['Values'].astype(str)

# Set the 'Values' column as the index of the DataFrame
max_column_names = max_column_names.set_index('Values')
max_column_names=max_column_names.assign(CellType='type')

# Remove rows where the index is "nan"
max_column_names = max_column_names[max_column_names.index!="nan"]

# For each row in the DataFrame, replace 'type' in the 'CellType' column with the actual cell type from the cell_type DataFrame
for i,cell_id in enumerate(max_column_names.index.values):
    max_column_names.iloc[i,-1]=cell_type.loc[cell_id][0]

# Reindex the coordinates_data DataFrame based on the 'Predict' column of the max_column_names DataFrame
# This gives the predicted coordinates for each cell
predict_coordinates=coordinates_data.reindex(max_column_names['Predict'])

# Add the predicted x and y coordinates to the max_column_names DataFrame
max_column_names['predict_x']=predict_coordinates['X'].values
max_column_names['predict_y']=predict_coordinates['Y'].values

# Print the DataFrame
print(max_column_names)

/tmp/ipykernel_3477685/2128575498.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  max_column_names.iloc[i,-1]=cell_type.loc[cell_id][0]


                             Predict    CellType  predict_x  predict_y
Values                                                                
GAATGAAAGGTAGCCA  AAACAAGTATCTCCCA-1  Nornal epi       2939       1190
TTATGCTAGGGTCGAT  AAACAAGTATCTCCCA-1  Nornal epi       2939       1190
GGATTACGTTTACTCT  AAACAAGTATCTCCCA-1      Clone1       2939       1190
GTGCGGTGTCATACTG  AAACAAGTATCTCCCA-1  Nornal epi       2939       1190
CGATGGCCAACGATGG  AAACAAGTATCTCCCA-1  Nornal epi       2939       1190
...                              ...         ...        ...        ...
TGGCTGGCAATAGAGT  TTGTTTGTGTAAATTC-1      Clone2        690       2693
AGAGCTTGTCTAGGTT  TTGTTTGTGTAAATTC-1     Myeloid        690       2693
TTCTTAGTCGGAGGTA  TTGTTTGTGTAAATTC-1     Myeloid        690       2693
AACACGTGTAGCGTCC  TTGTTTGTGTAAATTC-1      Clone2        690       2693
AGGCCGTAGCGATTCT  TTGTTTGTGTAAATTC-1      Clone1        690       2693

[7059 rows x 4 columns]


In [ ]:
assigned_locations_path = os.path.join(output_folder+"Cell2Map_DCIS2.csv")
max_column_names.to_csv(assigned_locations_path)